# Deep Learning with PyTorch



PyTorch is an open-source machine learning library based on the Torch library, developed by Facebook’s AI Research lab. It is widely used in deep learning, natural language processing, and computer vision applications. PyTorch provides a dynamic computational graph, which allows for easy and efficient modeling of complex neural networks.

Here’s a general overview of how to use PyTorch for deep learning:

Define the model: PyTorch provides a wide range of pre-built neural network architectures, such as fully connected networks, convolutional neural networks (CNNs), and recurrent neural networks (RNNs). The model can be defined using PyTorch’s torch.nn module.

Define the loss function: The loss function is used to evaluate the model’s performance and update its parameters. PyTorch provides a wide range of loss functions, such as cross-entropy loss and mean squared error loss.

Define the optimizer: The optimizer is used to update the model’s parameters based on the gradients of the loss function. PyTorch provides a wide range of optimizers, such as Adam and SGD.

Train the model: The model is trained on a dataset using the defined loss function and optimizer. PyTorch provides a convenient data loading and processing library, torch.utils.data, that can be used to load and prepare the data.

Evaluate the model: After training, the model’s performance can be evaluated on a test dataset.

Deploy the model: The trained model can be deployed in a wide range of applications, such as computer vision and natural language processing.

 neural net. 

Tensors: 


It turns out neural network computations are just a bunch of linear algebra operations on tensors, which are a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor. The fundamental data structure for neural networks are tensors and PyTorch is built around tensors. It’s time to explore how we can use PyTorch to build a simple neural network

In [5]:
!pip install torch torchvision torchaudio





In [7]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [8]:
#  1. Basic autograd example 1 
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


tensor(2.)
tensor(1.)
tensor(1.)


In [10]:
# 2. Basic autograd example 2 

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.5100, -0.3348, -0.0840],
        [ 0.3025, -0.2229,  0.4414]], requires_grad=True)
b:  Parameter containing:
tensor([-0.2538, -0.4422], requires_grad=True)
loss:  2.192253351211548
dL/dw:  tensor([[ 0.2596, -1.3067,  0.1537],
        [ 0.1219, -0.6320,  0.2301]])
dL/db:  tensor([-0.7705, -0.8655])
loss after 1 step optimization:  2.156494617462158


In [12]:
#   3. Loading data from numpy  

# Create a numpy array.
x = np.array([[1,2],[3,4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [14]:
# Mini-batch images and labels.
images, labels = next(data_iter)


import torch
import torchvision
import torchvision.transforms as transforms

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print(image.size())
print(label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = next(data_iter)

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass


Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        # Initialize file paths or a list of file names.
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        # Return the total number of files in the dataset.
        return len(self.image_paths)

    def __getitem__(self, index):
        # Read one data from file.
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        
        # Apply transformations if any.
        if self.transform:
            image = self.transform(image)
        
        # Use the filename or index as a placeholder for label.
        label = os.path.basename(image_path)  # Example label using filename.
        
        return image, label

# Define the image path.
image_path = [r'C:\Users\COMP\Desktop\face_rec\baby-boy.jpg']

# Define transformations (e.g., convert to tensor, normalize, etc.).
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize image to 32x32
    transforms.ToTensor(),  # Convert image to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image
])

# Initialize the dataset.
custom_dataset = CustomDataset(image_paths=image_path, transform=transform)

# Use DataLoader to handle batching (though with one image, batching is trivial).
train_loader = DataLoader(dataset=custom_dataset, batch_size=1, shuffle=False)

# Iterate through the dataset.
for images, labels in train_loader:
    # Print image tensor and label for verification.
    print(images.size(), labels)  # Image size and label (filename).


torch.Size([1, 3, 32, 32]) ('baby-boy.jpg',)


In [25]:
# PREtrain model



In [26]:
import torch
import torch.nn as nn
import torchvision.models as models

# ================================================================== #
#                         6. Pretrained Model                        #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass with a batch of random images.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())  # Expected output: (64, 100)

# ================================================================== #
#                      7. Save and Load the Model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
loaded_model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))


torch.Size([64, 100])


<All keys matched successfully>